In [1]:
import json
import numpy as np
from tqdm import tqdm
# import re

In [2]:
data_train = []
with open('PIZZA_train.json', 'r') as f:
    for line in f:
        data_train.append(json.loads(line))

print(len(data_train))

2456446


In [3]:
with open('PIZZA_dev.json', 'r') as file:
    data = json.load(file)

# Print the loaded data
print(data[0])

{'dev.SRC': 'i want to order two medium pizzas with sausage and black olives and two medium pizzas with pepperoni and extra cheese and three large pizzas with pepperoni and sausage', 'dev.EXR': '(ORDER (PIZZAORDER (NUMBER 2 ) (SIZE MEDIUM ) (COMPLEX_TOPPING (QUANTITY EXTRA ) (TOPPING CHEESE ) ) (TOPPING PEPPERONI ) ) (PIZZAORDER (NUMBER 2 ) (SIZE MEDIUM ) (TOPPING OLIVES ) (TOPPING SAUSAGE ) ) (PIZZAORDER (NUMBER 3 ) (SIZE LARGE ) (TOPPING PEPPERONI ) (TOPPING SAUSAGE ) ) )', 'dev.TOP': '(ORDER i want to order (PIZZAORDER (NUMBER two ) (SIZE medium ) pizzas with (TOPPING sausage ) and (TOPPING black olives ) ) and (PIZZAORDER (NUMBER two ) (SIZE medium ) pizzas with (TOPPING pepperoni ) and (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) ) ) and (PIZZAORDER (NUMBER three ) (SIZE large ) pizzas with (TOPPING pepperoni ) and (TOPPING sausage ) ) )', 'dev.PCFG_ERR': 'False'}


In [4]:
def extract_text_train(data):
    return data['train.SRC'], data['train.EXR'], data['train.TOP'], data['train.TOP-DECOUPLED']

In [5]:
def extract_text(data):
    return data['dev.SRC'], data['dev.EXR'], data['dev.TOP'], data['dev.PCFG_ERR']

In [6]:
sentences, explanations, topics, decoupled_topics = map(
    np.array, zip(*map(extract_text, data)))

In [8]:
sentences_t, explanations_t, topics_t, decoupled_topics_t = map(
    np.array, zip(*map(extract_text_train, data_train)))

In [9]:
entities = set([word[1:]
               for t in topics for word in t.split() if word.isupper()])

entities_train = set([word[1:]
               for t in topics_t for word in t.split() if word.isupper()])

full_entities = entities | entities_train

enitities_exclude_not = entities - {'NOT'}

negate_mapping = {}
for entity in entities:
    if entity != 'NOT':
        negate_mapping[entity] = "NOT_" + entity


In [10]:
print(entities)
print(entities_train)
print(full_entities)
print(enitities_exclude_not)
print(negate_mapping)

{'QUANTITY', 'TOPPING', 'PIZZAORDER', 'STYLE', 'NUMBER', 'COMPLEX_TOPPING', 'SIZE', 'DRINKORDER', 'DRINKTYPE', 'CONTAINERTYPE', 'NOT', 'ORDER'}
{'QUANTITY', 'TOPPING', 'PIZZAORDER', 'STYLE', 'NUMBER', 'COMPLEX_TOPPING', 'SIZE', 'DRINKORDER', 'DRINKTYPE', 'CONTAINERTYPE', 'VOLUME', 'NOT', 'ORDER'}
{'TOPPING', 'PIZZAORDER', 'NUMBER', 'SIZE', 'DRINKORDER', 'CONTAINERTYPE', 'VOLUME', 'QUANTITY', 'STYLE', 'COMPLEX_TOPPING', 'DRINKTYPE', 'NOT', 'ORDER'}
{'QUANTITY', 'TOPPING', 'PIZZAORDER', 'STYLE', 'NUMBER', 'COMPLEX_TOPPING', 'SIZE', 'DRINKORDER', 'DRINKTYPE', 'CONTAINERTYPE', 'ORDER'}
{'QUANTITY': 'NOT_QUANTITY', 'TOPPING': 'NOT_TOPPING', 'PIZZAORDER': 'NOT_PIZZAORDER', 'STYLE': 'NOT_STYLE', 'NUMBER': 'NOT_NUMBER', 'COMPLEX_TOPPING': 'NOT_COMPLEX_TOPPING', 'SIZE': 'NOT_SIZE', 'DRINKORDER': 'NOT_DRINKORDER', 'DRINKTYPE': 'NOT_DRINKTYPE', 'CONTAINERTYPE': 'NOT_CONTAINERTYPE', 'ORDER': 'NOT_ORDER'}


In [11]:
def negate_topic(topic):
    not_array = topic.split('NOT')
    not_array = not_array[1:]
    final_array = []
    for element in not_array:
        element = element.strip()
        open_brackets = 1
        result = ""
        index = 0
        while open_brackets > 0:
            if element[index] == '(':
                open_brackets += 1
            elif element[index] == ')':
                open_brackets -= 1
            if open_brackets > 0:
                result += element[index]
            index += 1

        final_array.append(result.strip())
    return final_array


def apply_negation(topic):
    array = negate_topic(topic)
    for el in array:
        original_el = el
        for entity in enitities_exclude_not:
            el = el.replace(entity, negate_mapping[entity])
            
        topic = topic.replace(original_el, el)
    return topic

In [12]:
negate_topic(topics[30])

['(TOPPING bacon )']

In [9]:
apply_negation(topics[30])

'(ORDER i want (PIZZAORDER (NUMBER a ) (TOPPING sausage ) pie with (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) ) no (NOT (NOT_TOPPING bacon ) ) ) )'

In [13]:
negated_topics = np.vectorize(apply_negation)(topics)

In [11]:
negated_topics[30]

np.str_('(ORDER i want (PIZZAORDER (NUMBER a ) (TOPPING sausage ) pie with (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) ) no (NOT (NOT_TOPPING bacon ) ) ) )')

In [12]:
topics[30]

np.str_('(ORDER i want (PIZZAORDER (NUMBER a ) (SIZE lunch size ) pizza with no (NOT (TOPPING apple wood bacon ) ) ) )')

In [13]:
print(len(entities))

13


In [16]:
entities

{'COMPLEX_TOPPING',
 'CONTAINERTYPE',
 'DRINKORDER',
 'DRINKTYPE',
 'NOT',
 'NUMBER',
 'ORDER',
 'PIZZAORDER',
 'QUANTITY',
 'SIZE',
 'STYLE',
 'TOPPING'}

In [30]:
full_entities

{'COMPLEX_TOPPING',
 'CONTAINERTYPE',
 'DRINKORDER',
 'DRINKTYPE',
 'NOT',
 'NUMBER',
 'ORDER',
 'PIZZAORDER',
 'QUANTITY',
 'SIZE',
 'STYLE',
 'TOPPING',
 'VOLUME'}

In [31]:
entities_copy = full_entities.copy()
entities_copy.remove('NOT')
entities_copy.remove('COMPLEX_TOPPING')
entities_copy.remove('PIZZAORDER')
entities_copy.remove('DRINKORDER')
entities_copy.remove('ORDER')
print(entities_copy)

{'QUANTITY', 'TOPPING', 'STYLE', 'NUMBER', 'SIZE', 'DRINKTYPE', 'CONTAINERTYPE', 'VOLUME'}


In [15]:
negate_mapping = {}
for entity in entities_copy:
    if entity != 'NOT':        negate_mapping[entity] = "NOT_" + entity

print(negate_mapping)

{'QUANTITY': 'NOT_QUANTITY', 'TOPPING': 'NOT_TOPPING', 'STYLE': 'NOT_STYLE', 'NUMBER': 'NOT_NUMBER', 'SIZE': 'NOT_SIZE', 'DRINKTYPE': 'NOT_DRINKTYPE', 'CONTAINERTYPE': 'NOT_CONTAINERTYPE', 'VOLUME': 'NOT_VOLUME'}


In [39]:
full_negate_entities= set(negate_mapping.values())|set(entities_copy)
np.save('full_negate_entities.npy', list(full_negate_entities))
print(full_negate_entities)

{'NOT_SIZE', 'SIZE', 'NOT_CONTAINERTYPE', 'VOLUME', 'NOT_STYLE', 'NOT_QUANTITY', 'TOPPING', 'CONTAINERTYPE', 'NOT_TOPPING', 'NOT_VOLUME', 'STYLE', 'NOT_DRINKTYPE', 'NOT_NUMBER', 'DRINKTYPE', 'NUMBER', 'QUANTITY'}


In [13]:
print(entities)

{'SIZE', 'DRINKORDER', 'TOPPING', 'ORDER', 'CONTAINERTYPE', 'PIZZAORDER', 'STYLE', 'COMPLEX_TOPPING', 'NOT', 'DRINKTYPE', 'NUMBER', 'QUANTITY'}


In [16]:
negated_entities = set([word[1:]
               for t in negated_topics for word in t.split() if word.isupper()])

In [33]:
len(negated_entities)

16

In [17]:
negated_entities

{'COMPLEX_NOT_TOPPING',
 'COMPLEX_TOPPING',
 'CONTAINERTYPE',
 'DRINKORDER',
 'DRINKTYPE',
 'NOT',
 'NOT_QUANTITY',
 'NOT_STYLE',
 'NOT_TOPPING',
 'NUMBER',
 'ORDER',
 'PIZZAORDER',
 'QUANTITY',
 'SIZE',
 'STYLE',
 'TOPPING'}

In [32]:
entities_copy = negated_entities.copy()
entities_copy.remove('COMPLEX_NOT_TOPPING')
entities_copy.remove('COMPLEX_TOPPING')
entities_copy.remove('PIZZAORDER')
entities_copy.remove('NOT')
entities_copy.remove('DRINKORDER')
entities_copy.remove('ORDER')
print(entities_copy)

{'TOPPING', 'NUMBER', 'SIZE', 'NOT_TOPPING', 'CONTAINERTYPE', 'NOT_QUANTITY', 'QUANTITY', 'STYLE', 'NOT_STYLE', 'DRINKTYPE'}


In [20]:
np.save('negated_entities.npy', list(entities_copy))

In [12]:
# tokens=np.load('tokenized_sentences_no_conct.npy', allow_pickle=True)
tokens=np.load('dev_tokenized_sentences_no_conct.npy', allow_pickle=True)

In [15]:
len(tokens)

2456446

In [13]:
tokens[0]

['i',
 'want',
 'to',
 'order',
 'two',
 'medium',
 'pizza',
 '##s',
 'with',
 'sausage',
 'and',
 'black',
 'olive',
 '##s',
 'and',
 'two',
 'medium',
 'pizza',
 '##s',
 'with',
 'pepper',
 '##oni',
 'and',
 'extra',
 'cheese',
 'and',
 'three',
 'large',
 'pizza',
 '##s',
 'with',
 'pepper',
 '##oni',
 'and',
 'sausage']

In [59]:
negated_topics[5]

np.str_('(ORDER (PIZZAORDER pie with (TOPPING banana pepper ) and (TOPPING peppperonis ) and (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING low fat cheese ) ) ) )')

In [70]:
# regex = re.compile(r'^[^\)]*\)')

# def extract_mapping(topic):
#     topic=topic.replace(
#     'COMPLEX_TOPPING', 'COMPLEX_T')
#     mapping={}
#     for entity in entities_copy:
#         splitted=topic.split(entity)
#         splitted=splitted[1:]
#         for el in splitted:
#             word=el.strip()
#             word=regex.search(word).group()[:-2]
#             for w in word.split():
#                 mapping[w]=entity
#     return mapping
            

# # extract_mapping(negated_topics[15])
# extract_mapping(
#     "(ORDER i need (PIZZAORDER (NUMBER a ) (SIZE large ) pizza and i want (TOPPING olives ) and (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) ) as well as (TOPPING chicken ) ) on it thanks a lot )")

{'extra': 'QUANTITY',
 'large': 'SIZE',
 'a': 'NUMBER',
 'olives': 'TOPPING',
 'cheese': 'TOPPING',
 'chicken': 'TOPPING'}

In [71]:
print(negated_entities)

{'QUANTITY', 'SIZE', 'DRINKORDER', 'PIZZAORDER', 'STYLE', 'ORDER', 'VOLUME', 'NOT_STYLE', 'CONTAINERTYPE', 'DRINKTYPE', 'NOT_TOPPING', 'NUMBER', 'TOPPING', 'COMPLEX_TOPPING', 'NOT'}


In [67]:
# def map_tokens_to_entities(tokens, mapping):
#     result = []  # List to store the token-entity pairs
#     buffer = ""  # Buffer to combine subwords

#     for token in tokens:
#         if token.startswith("##"):  # Handle subwords
#             buffer += token[2:]  # Append subword to the buffer
#         else:
#             if buffer:  # If there's a buffer, process it
#                 combined_word = buffer  # The completed word
#                 entity = mapping.get(combined_word, "O")  # Get entity or "O"
#                 result.append((combined_word, entity))
#                 buffer = ""  # Clear the buffer

#             buffer = token  # Start a new word with the current token if not a subword

#             # Directly map the token if it's not part of a subword
#             if not token.startswith("##"):
#                 entity = mapping.get(token, "O")
#                 result.append((token, entity))

#     # Process the last buffer if any
#     if buffer:
#         entity = mapping.get(buffer, "O")
#         result.append((buffer, entity))

#     # Ensure no duplicates
#     seen = set()
#     deduplicated_result = []
#     for word, entity in result:
#         if word not in seen:
#             seen.add(word)
#             deduplicated_result.append((word, entity))

#     return deduplicated_result




# print(tokens[15])
# pre_mapping=map_tokens_to_entities(tokens[15], extract_mapping(negated_topics[15]))
# for word, entity in pre_mapping:
#     print(f"{word} -> {entity}")


['i', 'would', 'like', 'a', 'party', 'sized', 'high', 'rise', 'dough', 'pie', 'with', 'a', 'lot', 'of', 'banana', 'pepper', 'and', 'pe', '##cor', '##ino', 'cheese']
i -> O
would -> O
like -> O
a -> NUMBER
party -> SIZE
sized -> SIZE
high -> STYLE
rise -> STYLE
dough -> STYLE
pie -> O
with -> O
lot -> QUANTITY
of -> QUANTITY
banana -> TOPPING
pepper -> TOPPING
and -> O
pe -> O
pecorino -> TOPPING
cheese -> TOPPING


In [132]:
# def get_entity_mapping(tokens,pre_mapping):
#     result=[]
#     continoued_mapping=None
#     for token, mapping in zip(reversed(tokens), reversed(pre_mapping)):
#         if continoued_mapping:
#             result.append((token,continoued_mapping))
#             if not token.startswith("##"):
#                 continoued_mapping=None
#         elif token.startswith("##"):
#             result.append((token,mapping[1]))
#             continoued_mapping=mapping[1]
#         else:
#             result.append((token, mapping[1]))
            
#     return list(reversed(result))

# get_entity_mapping(tokens[5],pre_mapping)

[('banana', 'O'),
 ('pepper', 'O'),
 ('and', 'TOPPING'),
 ('pep', 'O'),
 ('##pper', 'O'),
 ('##onis', 'O'),
 ('and', 'TOPPING'),
 ('extra', 'TOPPING'),
 ('low', 'TOPPING'),
 ('fat', 'TOPPING'),
 ('cheese', 'TOPPING')]

In [121]:
# for i,topic in enumerate(topics):
#     if(topic.count("NOT")>0):
#         print(i,topic)

In [33]:
def get_words_to_entity(topic,mapping):
    # print(mapping)
    words_array=topic.split()
    result=[]
    current_entity='O'
    for word in words_array:
        if word.startswith("("):
            if word[1:] in mapping:
                current_entity=word[1:]
            else:
                current_entity='O'
        elif word.startswith(")"):
            current_entity='O'
        else:
            result.append((word,current_entity))
    
    return result
        


mp=get_words_to_entity(negated_topics[200],entities_copy)
print(mp)
# print(tokens[500])

[('let', 'O'), ('me', 'O'), ('have', 'O'), ('an', 'O'), ('order', 'O'), ('of', 'O'), ('two', 'NUMBER'), ('small', 'SIZE'), ('pizzas', 'O'), ('with', 'O'), ('black', 'TOPPING'), ('olives', 'TOPPING'), ('and', 'O'), ('jalapeno', 'TOPPING'), ('peppers', 'TOPPING')]


In [34]:
entities_copy

{'CONTAINERTYPE',
 'DRINKTYPE',
 'NOT_QUANTITY',
 'NOT_STYLE',
 'NOT_TOPPING',
 'NUMBER',
 'QUANTITY',
 'SIZE',
 'STYLE',
 'TOPPING'}

In [35]:
def get_entity(topic,mapping):
    # print(mapping)
    words_array=topic.split()
    result=[]
    current_entity='O'
    first=False
    for word in words_array:
        if word.startswith("("):
            if word[1:] in mapping:
                current_entity=word[1:]
                first=True
            else:
                current_entity='O'
        elif word.startswith(")"):
            current_entity='O'
        else:
            if current_entity=='O':
                result.append(current_entity)
            else:
                if first:
                    result.append('B-'+current_entity)
                    first=False
                else:
                    result.append('I-'+current_entity)    
    return result
        

number=300
print(negated_topics[number])
mp=get_entity(negated_topics[number],entities_copy)
print(mp)
# print(tokens[500])

(ORDER can i have (PIZZAORDER (NUMBER three ) pizzas with (TOPPING peppers ) and (TOPPING chicken ) but without adding (NOT (NOT_TOPPING ham ) ) ) )
['O', 'O', 'O', 'B-NUMBER', 'O', 'O', 'B-TOPPING', 'O', 'B-TOPPING', 'O', 'O', 'O', 'B-NOT_TOPPING']


In [19]:
print(negated_topics[200])

(ORDER let me have an order of (PIZZAORDER (NUMBER two ) (SIZE small ) pizzas with (TOPPING black olives ) and (TOPPING jalapeno peppers ) ) )


In [21]:
entities_to_words_not_processed=np.vectorize(get_entity)(negated_topics,entities_copy)

ValueError: setting an array element with a sequence

In [22]:
def get_tokens_to_entity(tokens,topic,entities):
    mp = get_words_to_entity(topic,entities)
    token_idx=len(tokens)-1
    mp_idx=len(mp)-1
    mp_entities=[entity for word,entity in mp]
    result=[]
    current_entity=None
    while token_idx>=0:
        t=tokens[token_idx]
        if current_entity != None:
            result.append((t,current_entity))
            if not t.startswith("##"):
                current_entity=None
                mp_idx-=1
        elif t.startswith("##"):
            result.append((t,mp_entities[mp_idx]))
            current_entity=mp_entities[mp_idx]
        else:
            result.append((t,mp_entities[mp_idx]))
            mp_idx-=1
        token_idx-=1
    return list(reversed(result))
            
            
    

get_tokens_to_entity(tokens[200],negated_topics[200],entities_copy)

[('let', 'O'),
 ('me', 'O'),
 ('have', 'O'),
 ('an', 'O'),
 ('order', 'O'),
 ('of', 'O'),
 ('two', 'NUMBER'),
 ('small', 'SIZE'),
 ('pizza', 'O'),
 ('##s', 'O'),
 ('with', 'O'),
 ('black', 'TOPPING'),
 ('olive', 'TOPPING'),
 ('##s', 'TOPPING'),
 ('and', 'O'),
 ('ja', 'TOPPING'),
 ('##la', 'TOPPING'),
 ('##pen', 'TOPPING'),
 ('##o', 'TOPPING'),
 ('peppers', 'TOPPING')]

In [23]:

tokens_to_entities = [
    get_tokens_to_entity(tokens,topic,entities_copy) for tokens,topic in tqdm(zip(tokens,negated_topics), desc="Mapping tokens to entities")
]

Mapping tokens to entities: 348it [00:00, 33399.34it/s]


In [24]:
tokens_to_entities=np.array(tokens_to_entities, dtype=object)

In [25]:
# np.save('tokens_to_entities.npy', tokens_to_entities)
np.save('dev_tokens_to_entities.npy', tokens_to_entities)

In [29]:
len(tokens_to_entities)

2456446

In [30]:
tokens_to_entities[0]

[('can', 'O'),
 ('i', 'O'),
 ('have', 'O'),
 ('a', 'NUMBER'),
 ('large', 'SIZE'),
 ('bb', 'TOPPING'),
 ('##q', 'TOPPING'),
 ('pulled', 'TOPPING'),
 ('pork', 'TOPPING')]